## Process Utilization Files and Generate a Pickle Files for Plotting

### Note that running following scripts is time consuming and should be done only once. The pickle files are provided in the `util_pkl` directory.

In [1]:
import pytz
import pickle
import numpy as np
import pandas as pd

from glob import glob
from pathlib import Path
from utils import cluster_metric_header, dcgm_metric_header

""" cluster_metric_header
0: "CPU",
1: "MEMORY",
2: "IB_SEND",
3: "RECEIVE",
"""
""" dcgm_metric_header
0: XID_ERRORS
1: GPU_TEMP
2: MEMORY_TEMP
3: MEM_CLOCK
4: MEM_COPY_UTIL
5: FB_FREE
6: FB_USED
7: DRAM_ACTIVE
8: POWER_USAGE
9: GPU_UTIL
10: PIPE_TENSOR_ACTIVE
11: SM_ACTIVE
12: SM_OCCUPANCY
"""

SAVEPKL = "./util_pkl"


def read_csv_with_concat(path="./csv", file_name=None):
    file = Path(path, f"{file_name}.csv")

    if file.exists():
        # If original file exists, read it directly
        df = pd.read_csv(file)
        print(f"Reading {file_name}")
    else:
        # If original file does not exist, read all the split files
        split_files = sorted(glob(f"{path}/{file_name}-2023-*.csv"))
        print(f"Reading splitted files: {split_files}")
        df = pd.concat([pd.read_csv(split_file) for split_file in split_files])
        df.reset_index(drop=True, inplace=True)
    return df


def read_concat_parse_save_cluster_metrics(path="./csv_cpu", metrics=cluster_metric_header):
    for metric in metrics:
        data = read_csv_with_concat(path=path, file_name=metric)
        data.drop_duplicates(subset=["Time"], inplace=True)
        data.sort_values(by="Time", inplace=True)
        data["Time"] = pd.to_datetime(data["Time"], unit="s").dt.tz_localize(pytz.utc).dt.tz_convert("Asia/Shanghai")
        data.set_index("Time", drop=True, inplace=True)
        print(f"Column Number: {len(list(data.columns))}, {len(set(list(data.columns)))}")

        if "NODE_MEMORY" in metric:
            # Around 2 hours has some bug (ip has additional '.1', like '10.140.0.131' -> '10.140.0.131.1')
            data = data[(data.index < "2023-07-19 11:35:00") | (data.index > "2023-07-19 14:01:00")]

        if "NODE_CPU" in metric or "NODE_MEMORY" in metric:
            data = data * 100  # CPU / Memory Utilization (%)

        if "NODE_IB" in metric:
            data.rename(columns=lambda x: x.replace("-mlx5_0", ""), inplace=True)  # Simplified, since one IB NIC per server

        data.dropna(axis=1, how="all", inplace=True)
        data = data.round(3)
        data.to_csv(f"./{metric}.csv")


def read_concat_parse_save_dcgm_metrics(path="./csv", metrics=dcgm_metric_header):
    for metric in metrics:
        data = read_csv_with_concat(path=path, file_name=metric)
        data.drop_duplicates(subset=["Time"], inplace=True)
        data.sort_values(by="Time", inplace=True)
        data["Time"] = pd.to_datetime(data["Time"], unit="s").dt.tz_localize(pytz.utc).dt.tz_convert("Asia/Shanghai")
        data.set_index("Time", drop=True, inplace=True)
        print(f"Column Number: {len(list(data.columns))}, {len(set(list(data.columns)))}")

        # if "XID" in metric or "TEMP" in metric or "CLOC" in metric:
        #     data = data.astype(int, errors='ignore')

        if "ACTIVE" in metric or "OCCUPANCY" in metric:
            data = data * 100  # CPU / Memory Utilization (%)
            data = data.round(3)

        if "POWER" in metric:
            data = data.round(1)

        data.dropna(axis=0, how="all", inplace=True)
        data.dropna(axis=1, how="all", inplace=True)
        data.to_csv(f"./{metric}.csv")


def calculate_sum_cdf_axis100(df, dot_num=1000):
    """
    Calculate quantity percentile CDF, y-axis: 0-100%,
    """
    print("Parsing")
    data = df.melt(id_vars="Time", var_name="Server")
    data.dropna(subset=["value"], inplace=True)

    y = np.linspace(0, 1, num=dot_num)
    x = data["value"].quantile(y).values
    y = y * 100
    return x, y


def calculate_num_cdf_axis100(df, dot_num=1000):
    """
    Calculate quantity percentile CDF, y-axis: 0-100%,
    """
    print("Parsing")
    data = df.melt(id_vars="Time", var_name="Server")
    data.dropna(subset=["value"], inplace=True)
    # data.sort_values('value', ascending=True, inplace=True)
    # data.reset_index(drop=True, inplace=True)

    y = np.linspace(0, 1, num=dot_num)
    x = data["value"].quantile(y).values
    y = y * 100
    return x, y

### Example 1: Prometheus Metics (e.g., CPU and Memory Utilization)

You can change to any metric you want to plot by changing the `file_name` variable in the following script.

In [ ]:
data_cpu = read_csv_with_concat(path="./seren", file_name=cluster_metric_header[0])
data_mem = read_csv_with_concat(path="./seren", file_name=cluster_metric_header[1])
x1, y1 = calculate_num_cdf_axis100(data_cpu)
x2, y2 = calculate_num_cdf_axis100(data_mem)
print(
    f'CPU Period: (Start) {data_cpu.at[0, "Time"].split(":")[0]}h (End) {data_cpu.at[len(data_cpu)-1, "Time"].split(":")[0]}h'
)
print(
    f'MEM Period: (Start) {data_mem.at[0, "Time"].split(":")[0]}h (End) {data_mem.at[len(data_mem)-1, "Time"].split(":")[0]}h'
)

with open(f"{SAVEPKL}/util_cpu_mem_seren.pkl", "wb") as file:
    pickle.dump([x1, y1, x2, y2], file)

### Example 2: NVIDIA DCGM Metics (e.g., GPU and GPU Memory Utilization)

You can change to any metric you want to plot by changing the `file_name` variable in the following script.

In [ ]:
data_gpu_util = read_csv_with_concat(path="./seren", file_name=dcgm_metric_header[9])  # "DCGM_FI_DEV_GPU_UTIL"
data_gpu_mem = read_csv_with_concat(path="./seren", file_name=dcgm_metric_header[6])  # "DCGM_FI_DEV_FB_USED"
data_gpu_mem.iloc[:, 1:] = 100 * data_gpu_mem.iloc[:, 1:] / (80 * 1024)
x1, y1 = calculate_num_cdf_axis100(data_gpu_util)
x2, y2 = calculate_num_cdf_axis100(data_gpu_mem)

with open(f"{SAVEPKL}/util_gpu_util_mem_seren.pkl", "wb") as file:
    pickle.dump([x1, y1, x2, y2], file)

### Processing IPMI Power Files

In [ ]:
df_AB = pd.read_csv("./ipmi/GPU_AB_Power.csv", parse_dates=["Time"])
df_C = pd.read_csv("./ipmi/GPU_C_Power.csv", parse_dates=["Time"])
df_D = pd.read_csv("./ipmi/CPU_D_Power.csv", parse_dates=["Time"])

df_A = df_AB.dropna()
df_B = df_AB[df_AB.isna().any(axis=1)]  # Type B without MEM_Power record

dfs = {"GPU_A": df_A, "GPU_B": df_B, "GPU_C": df_C, "CPU_D": df_D}

# Extract sys_total_power
df_A_power = df_A[["Time", "Sys_Total_Power"]]
df_B_power = df_B[["Time", "Sys_Total_Power"]]
df_C_power = df_C[["Time", "Sys_Total_Power"]]
df_gpu = pd.concat([df_A_power, df_B_power, df_C_power])

x1, y1 = calculate_sum_cdf_axis100(df_gpu)
x2, y2 = calculate_sum_cdf_axis100(df_D[["Time", "Sys_Total_Power"]])

with open(f"./server_power.pkl", "wb") as file:
    pickle.dump([x1, y1, x2, y2], file)

### Processing Philly GPU Utilization Data

In [45]:
philly = pd.read_csv(
    "./philly/philly_gpu_util.csv", on_bad_lines="skip", header=0
)  # Please refer to their official repo for the data
cols = list(philly.columns)
philly = philly.drop(columns=[cols[-1]])
philly.reset_index(inplace=True)
philly.columns = cols
philly.rename(columns={"time": "Time"}, inplace=True)
philly = philly.drop(columns=[cols[1]])

x1, y1 = calculate_num_cdf_axis100(philly)
with open(f"{SAVEPKL}/util_gpu_util_philly.pkl", "wb") as file:
    pickle.dump([x1, y1], file)